<a href="https://colab.research.google.com/github/Navifra-Denny/01_linear_algegra/blob/main/01.pca/principal_component_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

In [2]:
SLOPE_MIN = -50.0
SLOPE_MAX = 50.0

# ===== 1. Îç∞Ïù¥ÌÑ∞ ÏÉùÏÑ± Ìï®Ïàò =====
def generate_clipped_gaussian(mean, std, min_val, max_val, size):
    data = np.random.normal(mean, std, size)
    data = np.clip(data, min_val, max_val)
    return np.round(data, 1)

In [19]:
# ===== 2. PCA Ï£ºÏÑ±Î∂Ñ Î∞©Ìñ• Í≥ÑÏÇ∞ =====
def calculate_pca_vectors(x, y):
    data = np.column_stack((x, y))
    mean = np.mean(data, axis=0)
    centered = data - mean
    cov = np.cov(centered, rowvar=False)
    eigvals, eigvecs = np.linalg.eigh(cov)  # ascending order
    pc1 = eigvecs[:, 1]  # largest eigenvalue
    pc2 = eigvecs[:, 0]  # smallest eigenvalue (orthogonal)
    return pc1, pc2, mean

## SS (Sumo of Squares)

![ss](https://raw.githubusercontent.com/Navifra-Denny/01_linear_algegra/main/01.pca/img.png)


> Î™®Îì† Îç∞Ïù¥ÌÑ∞ÏóêÏÑú ÏõêÏ†ê(ÌèâÍ∑†ÏúºÎ°ú Ïù¥ÎèôÎêú Ï†ê)ÏùÑ ÏßÄÎÇòÎäî ÏßÅÏÑ†Ïóê ÏàòÏÑ†ÏùÑ Î∞úÏùÑ ÎÇ¥Î¶¨Î©¥, ÏõêÏ†êÏúºÎ°úÎ∂ÄÌÑ∞ ÏàòÏÑ†Ïùò Î∞úÍπåÏßÄÏùò Í∏∏Ïù¥Î•º Íµ¨Ìï† Ïàò ÏûàÍ≤å ÎêúÎã§. ÏõêÏ†êÏùÑ ÏßÄÎÇòÎäî ÏßÅÏÑ†Ïùò Í∏∞Ïö∏Í∏∞Í∞Ä Î≥ÄÌï®Ïóê Îî∞Îùº, Ïù¥ Îπ®Í∞ÑÏÑ†Îì§Ïùò Í∏∏Ïù¥ ÎòêÌïú Î≥ÄÌïòÍ≤å ÎêúÎã§. PCA ÏóêÏÑúÎäî Ïù¥ Îπ®Í∞ÑÏÑ†Îì§Ïùò Í∏∏Ïù¥ Ï†úÍ≥±Îì§Ïùò Ìï©Ïù¥ ÏµúÎåÄÍ∞Ä ÎêòÎäî ÏßÅÏÑ†ÏùÑ Ï∞æÎäîÎã§. Ïù¥Í≤ÉÏù¥ SS(Sum of Squares) Ïù¥Îã§.


## ÏöîÏïΩ
* ÏßÅÏÑ†ÏùÄ ÌèâÍ∑†Ï†êÏùÑ ÏßÄÎÇòÎ©∞, Í∏∞Ïö∏Í∏∞ ùëö Ïóê Îî∞Îùº Ï†ïÏùòÎê®:
$$ y = m(x - \bar{x}) + \bar{y} $$
* Í∞Å Îç∞Ïù¥ÌÑ∞ Ìè¨Ïù∏Ìä∏ $(x_i, y_i)$ Ïóê ÎåÄÌï¥ ÏßÅÏÑ†Ïóê ÎÇ¥Î¶∞ **ÏàòÏÑ†Ïùò Î∞ú**ÏùÑ Íµ¨ÌïòÍ≥†,
* Í∑∏ **ÏàòÏÑ† Í∏∏Ïù¥Ïùò Ï†úÍ≥±**ÏùÑ Í≥ÑÏÇ∞Ìïú ÌõÑ, Ï†ÑÏ≤¥Ïóê ÎåÄÌïú Ìï©ÏÇ∞:
$$ SS(m) = \sum_{i=1}^{N} d_i^2 $$


### ÏßÅÏÑ†Í≥º Ï†ê ÏÇ¨Ïù¥Ïùò Í±∞Î¶¨ Í≥ÑÏÇ∞ Í≥µÏãù
* ÏßÅÏÑ†: $y=mx+b$
* Ï†ê: $(x_0, y_0)$
* ÏßÅÏÑ†Í≥º Ï†ê ÏÇ¨Ïù¥ Í±∞Î¶¨:
$$ d = \frac{\left| mx_0 - y_0 + b \right|}{\sqrt{m^2 + 1}} $$


In [20]:
# ===== 3. SS Í≥ÑÏÇ∞ Ìï®Ïàò (Î≤°ÌÑ∞ Ìà¨ÏòÅ Í∏∞Î∞ò) =====
def calculate_ss_from_vector(x, y, direction_vector):
    data = np.column_stack((x, y))
    mean = np.mean(data, axis=0)
    centered = data - mean

    # ÏàòÏßÅ Î∞©Ìñ• Î≤°ÌÑ∞ Íµ¨ÌïòÍ∏∞
    perp_vector = np.array([-direction_vector[1], direction_vector[0]])
    perp_vector = perp_vector / np.linalg.norm(perp_vector)

    projections = centered @ perp_vector
    ss = np.sum(projections**2)
    return ss


In [21]:
# ===== 4. ÏÇ¨Ïö©Ïûê Í∏∞Ïö∏Í∏∞Î•º Î≤°ÌÑ∞Î°ú Î≥ÄÌôò =====
def slope_to_unit_vector(slope):
    vec = np.array([1.0, slope])
    return vec / np.linalg.norm(vec)

In [22]:
# num_samples = 300

# # Gaussian parameters
# height_mean, height_std = 170, 6.8
# weight_mean, weight_std = 80, 13

# # Generate height and weight with clipping and rounding
# heights = generate_clipped_gaussian(height_mean, height_std, 0, 210, num_samples)
# weights = generate_clipped_gaussian(weight_mean, weight_std, 40, 150, num_samples)

# # Stack into (N, 2) dataset
# dataset = np.column_stack((heights, weights))

# # Compute means
# height_avg = np.mean(heights)
# weight_avg = np.mean(weights)


In [27]:
# ===== 5. Îç∞Ïù¥ÌÑ∞ ÏÉùÏÑ± Î∞è PCA Î≤°ÌÑ∞ Í≥ÑÏÇ∞ =====
num_samples = 300
heights = generate_clipped_gaussian(170, 17, 0, 300, num_samples)
weights = generate_clipped_gaussian(80, 13, 40, 150, num_samples)

pca_vec1, pca_vec2, center = calculate_pca_vectors(heights, weights)

In [32]:
# ===== 6. ÏãúÍ∞ÅÌôî ÏóÖÎç∞Ïù¥Ìä∏ Ìï®Ïàò =====
def update_plot(m):
    user_vec = slope_to_unit_vector(m)
    ss_user = calculate_ss_from_vector(heights, weights, user_vec)
    ss_pca = calculate_ss_from_vector(heights, weights, pca_vec1)
    ss_ortho = calculate_ss_from_vector(heights, weights, pca_vec2)

    plt.figure(figsize=(8, 6))
    plt.scatter(heights, weights, color='skyblue', edgecolors='black', label='Data')

    # ÌèâÍ∑†Ï†ê
    plt.axvline(x=center[0], color='red', linestyle='--', linewidth=0.5)
    plt.axhline(y=center[1], color='red', linestyle='--', linewidth=0.5)

    # ÏÇ¨Ïö©Ïûê ÏßÅÏÑ†
    x_vals = np.linspace(center[0] - 50, center[0] + 50, 100)
    y_user = m * (x_vals - center[0]) + center[1]
    plt.plot(x_vals, y_user, color='green', label=f'Slope={m:.2f} | SS={ss_user:.1f}', linewidth=2)

    # PCA 1 (Î∂ÑÏÇ∞ ÏµúÎåÄ Î∞©Ìñ•)
    slope_pca = pca_vec1[1] / pca_vec1[0]
    y_pca = slope_pca * (x_vals - center[0]) + center[1]
    plt.plot(x_vals, y_pca, color='orange', linestyle='--', linewidth=2,
             label=f'PCA 1 (SS={ss_pca:.1f})')

    # PCA 2 (ÏßÅÍµê Î∞©Ìñ•)
    slope_ortho = pca_vec2[1] / pca_vec2[0]
    y_ortho = slope_ortho * (x_vals - center[0]) + center[1]
    plt.plot(x_vals, y_ortho, color='purple', linestyle='--', linewidth=2,
             label=f'Ortho (SS={ss_ortho:.1f})')

    # Î≥¥Í∏∞ Ìé∏Ìïú Ï∂ï Î≤îÏúÑ
    plt.xlim(center[0] - 40, center[0] + 40)
    plt.ylim(center[1] - 40, center[1] + 40)
    plt.title("PCA vs Perpendicular Distance (SS)")
    plt.xlabel("Height (cm)")
    plt.ylabel("Weight (kg)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [33]:
# Ïä¨ÎùºÏù¥Îçî: Í∏∞Ïö∏Í∏∞ m (-5 ~ 5 ÏÇ¨Ïù¥ Ïã§Ïàò)
slope_slider = widgets.FloatSlider(
    value=0.0,
    min=SLOPE_MIN,
    max=SLOPE_MAX,
    step=0.1,
    description='Slope (m):',
    continuous_update=True
)

widgets.interact(update_plot, m=slope_slider);

interactive(children=(FloatSlider(value=0.0, description='Slope (m):', max=50.0, min=-50.0), Output()), _dom_c‚Ä¶